In [10]:
#example script demonstrating Small-Fry API usage
import smallfry as sfry

In [11]:
#Inputs for the compressor
source_path = "glove.head.txt" #100 lines out of the offical glove.6B embeddings
prior_path = "prior.npy" #A prior for these 100 words in dict format saved as npy. Maps word => numeric.
word_rep = "trie" #Let's use the marisa-trie representation for the word list -- it's more compact that a dict!

#NOTE -- space constraint: 
#We are going to use the standard bitrate, R = 1, for compression.
#If you have more space for your embeddings, be sure to give yourself a looser memory constraint!
#You may specify either as an approximate memory budget (in bytes) or as a bitrate (avg bits per entry in matrix)

#NOTE -- output directory:
#You may optionally specify an output directory using the param outdir="<myoutputdirectorypath>".
#Otherwise, Small-Fry's compression will be written to the same directory as the source embeddings.

In [12]:
#Make the API call
word2idx, sfry_path = sfry.compress(source_path, prior_path, word_rep=word_rep)

#NOTE -- word representation:
#The user is responsible for keeping track of the word2idx. Setting the <write_word_rep> flag will
#automatically write the representation to the output directory.
#If the flag is not set, the representation is still returned from the API call, and a text word list is written 
#to the output directory instead

AttributeError: module 'smallfry' has no attribute 'compress'

In [ ]:
#Query Small-Fry for your word!

vector = sfry.query('them', word2idx, sfry_path) #returns a numpy vector

print(vector)

#You may also be interested in loading Small-Fry into memory to speed querying up

my_smallfry = sfry.load(sfry_path, word2idx) #generates a wrapper object for memory-mapped Small-Fry

my_smallfry.query*

